In [1]:
import random
import typing as _t
from pathlib import Path

import instructor
import openai
from dotenv import load_dotenv
from pydantic import AfterValidator, BaseModel, Field

from dreamai.ai import ModelName, system_message, user_message

random.seed(42)

load_dotenv()

ask_oai = instructor.from_openai(openai.OpenAI())

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
ATTEMPTS = 3

MIN_SUBTOPICS = 2
MAX_SUBTOPICS = 4
MIN_CONCEPTS = 2
MAX_CONCEPTS = 4


# The CreatedSubtopic class is a Pydantic model that represents a subtopic.
# Pydantic models are used for data validation, serialization, and documentation.
class CreatedSubtopic(BaseModel):
    # The name of the subtopic. It is a string and will be transformed to title case after validation.
    name: _t.Annotated[str, AfterValidator(str.title)]
    # The concepts covered in the subtopic. It is a list of strings.
    # Each string will be transformed to title case after validation.
    # The number of concepts should be between MIN_CONCEPTS and MAX_CONCEPTS. If they are not, our AI will try again. Up to ATTEMPTS times.
    concepts: _t.Annotated[
        list[str], AfterValidator(lambda concepts: [x.title() for x in concepts])
    ] = Field(
        f"{MIN_CONCEPTS}-{MAX_CONCEPTS} concepts covered in the subtopic.",
        min_length=MIN_CONCEPTS,
        max_length=MAX_CONCEPTS,
    )


# The CreatedTopic class is a Pydantic model that represents a topic.
class CreatedTopic(BaseModel):
    # The name of the topic. It is a string and will be transformed to title case after validation.
    name: _t.Annotated[str, AfterValidator(str.title)]
    # The subtopics of the topic. It is a list of CreatedSubtopic objects.
    # The number of subtopics should be between MIN_SUBTOPICS and MAX_SUBTOPICS. If they are not, our AI will try again. Up to ATTEMPTS times.
    subtopics: list[CreatedSubtopic] = Field(
        f"{MIN_SUBTOPICS}-{MAX_SUBTOPICS} ordered subtopics with concepts.",
        min_length=MIN_SUBTOPICS,
        max_length=MAX_SUBTOPICS,
    )

In [3]:
COURSE = "math_102"
SYSTEM_PREFIX = "You are a world class math course instructor."

In [4]:
outline = Path(f"{COURSE}.txt").read_text().splitlines()
outline

['Set, its different representations and types of sets',
 'Complex numbers their addition, subtraction, multiplication and division and Modulus of a complex number',
 'Mapping and their types, function and their types, composite and inverse of a function, addition, subtraction, multiplication and division of functions',
 'Quadratic functions and quadratic formula with the types of solutions',
 'Matrices with their addition, subtraction  and multiplication , finding the inverse of a matrix using augmented matrix and co-factors. Application of matrices in solving system of linear equations, Crammer rule Determinants',
 'Arithmetic and Geometric Sequence with their nth terms and Series sum.',
 'Permutations and Combinations with their applications on some real life scenarios',
 'Binomial theorem and its applications in generalizing the formulas for higher powers ',
 'In coordinate geometry find  the distance between two points, the slope of a line,condition for parallel and perpendicular 

In [8]:
topic_system = f"""\
{SYSTEM_PREFIX}
You'll be given a topic description from a course outline and you have to generate a 3-5 word topic name that encapsulates the description.
Then, generate {MIN_SUBTOPICS}-{MAX_SUBTOPICS} subtopics for the topic. Also 3-5 words each.
Then for each subtopic, generate {MIN_CONCEPTS}-{MAX_CONCEPTS} concepts. Also 3-5 words each. The concepts should be related to the subtopic.
Think of concepts as the smallest unit of knowledge that can be taught from the subtopic.
"""

# Let's show an example user message and also the system message
line = outline[0]
user_msg = user_message(
    f"<topic_description>\n{line}\n</topic_description>"
)  # this is a dict with a "user" role and some content
print(user_msg["content"])
print()
print(user_msg)
print()
print(topic_system)

<topic_description>
Set, its different representations and types of sets
</topic_description>

{'role': 'user', 'content': '<topic_description>\nSet, its different representations and types of sets\n</topic_description>'}

You are a world class math course instructor.
You'll be given a topic description from a course outline and you have to generate a 3-5 word topic name that encapsulates the description.
Then, generate 2-4 subtopics for the topic. Also 3-5 words each.
Then for each subtopic, generate 2-4 concepts. Also 3-5 words each. The concepts should be related to the subtopic.
Think of concepts as the smallest unit of knowledge that can be taught from the subtopic.



In [9]:
topics = []  # this will be a list of CreatedTopic objects
for line in outline:
    print(f"\n\nCREATING TOPIC FOR LINE: {line}\n\n")
    topics.append(
        ask_oai.create(
            response_model=CreatedTopic,
            messages=[
                system_message(topic_system),
                user_message(f"<topic_description>\n{line}\n</topic_description>"),
            ],  # type: ignore
            model=ModelName.GPT_4,
            max_retries=ATTEMPTS,
        )
    )



CREATING TOPIC FOR LINE: Set, its different representations and types of sets




CREATING TOPIC FOR LINE: Complex numbers their addition, subtraction, multiplication and division and Modulus of a complex number




CREATING TOPIC FOR LINE: Mapping and their types, function and their types, composite and inverse of a function, addition, subtraction, multiplication and division of functions




CREATING TOPIC FOR LINE: Quadratic functions and quadratic formula with the types of solutions




CREATING TOPIC FOR LINE: Matrices with their addition, subtraction  and multiplication , finding the inverse of a matrix using augmented matrix and co-factors. Application of matrices in solving system of linear equations, Crammer rule Determinants




CREATING TOPIC FOR LINE: Arithmetic and Geometric Sequence with their nth terms and Series sum.




CREATING TOPIC FOR LINE: Permutations and Combinations with their applications on some real life scenarios




CREATING TOPIC FOR LINE: Binomial theo

In [10]:
topics

[CreatedTopic(name='Set Theory Basics', subtopics=[CreatedSubtopic(name='Set Representations', concepts=['Roster Notation', 'Set-Builder Notation', 'Venn Diagrams']), CreatedSubtopic(name='Types Of Sets', concepts=['Finite Sets', 'Infinite Sets', 'Null Sets', 'Equal Sets']), CreatedSubtopic(name='Set Operations', concepts=['Union And Intersection', 'Difference Of Sets', 'Complement Of Sets', 'Cartesian Products'])]),
 CreatedTopic(name='Complex Numbers', subtopics=[CreatedSubtopic(name='Basic Operations', concepts=['Addition', 'Subtraction', 'Multiplication', 'Division']), CreatedSubtopic(name='Modulus Of Complex Numbers', concepts=['Definition Of Modulus', 'Geometric Representation', 'Calculating Modulus'])]),
 CreatedTopic(name='Functions And Mappings', subtopics=[CreatedSubtopic(name='Types Of Mappings', concepts=['One-To-One Mapping', 'Onto Mapping', 'Many-To-One Mapping']), CreatedSubtopic(name='Types Of Functions', concepts=['Linear Functions', 'Quadratic Functions', 'Polynomial 

In [11]:
# In Pydantic, we can access the data as a dict using the .model_dump() method.
topics[0].model_dump()

{'name': 'Set Theory Basics',
 'subtopics': [{'name': 'Set Representations',
   'concepts': ['Roster Notation', 'Set-Builder Notation', 'Venn Diagrams']},
  {'name': 'Types Of Sets',
   'concepts': ['Finite Sets', 'Infinite Sets', 'Null Sets', 'Equal Sets']},
  {'name': 'Set Operations',
   'concepts': ['Union And Intersection',
    'Difference Of Sets',
    'Complement Of Sets',
    'Cartesian Products']}]}